In [62]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing import pipeline2

In [63]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 데이터 전처리

In [64]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
df = pd.read_csv(file_path)

# 데이터 전처리
preprocessing_pipeline = pipeline(type='train')
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250320-01"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_8608\3777277752.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\developments\hk-project\src\data_preprocessing.py:146: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(csv_path, encoding="cp949")
c:\Users\user\developments\hk-project\src\data_preprocessing.py:330: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original 

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1870 entries, 0 to 9362
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        1870 non-null   int64  
 1   공급규모          1870 non-null   int64  
 2   투기과열지구        1870 non-null   object 
 3   조정대상지역        1870 non-null   object 
 4   분양가상한제        1870 non-null   object 
 5   정비사업          1870 non-null   object 
 6   공공주택지구        1870 non-null   object 
 7   대규모택지개발지구     1870 non-null   object 
 8   수도권내민영공공주택지구  1870 non-null   object 
 9   공급세대수         1870 non-null   int64  
 10  순위            1870 non-null   object 
 11  거주지역          1870 non-null   object 
 12  접수건수          1870 non-null   int64  
 13  경쟁률           1870 non-null   float64
 14  최저당첨가점        1870 non-null   float64
 15  토픽 1          1870 non-null   float64
 16  토픽 2          1870 non-null   float64
 17  토픽 3          1870 non-null   float64
 18  토픽 4          1870 non-null   flo

### 모델 학습 및 평가

In [66]:
# 전처리된 데이터 가져오기

file_path = f"./storage/train_data/train-{file_version}.csv"
df = pd.read_csv(file_path, encoding='cp949')

# df = pd.read_csv("./storage/train_data/train-2트_시세차익o.csv", encoding = 'cp949')

In [67]:
df

,공급지역코드,공급규모,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,공급세대수,...,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,전용면적,공급금액(최고가 기준),법정동코드,거래금액(만원),전용면적당 시세차익
0,410,64,N,N,N,N,N,N,N,5,...,0.056333,0.045233,0.175067,0.363900,0.296767,60.9590,5.075000e+08,4.111113e+09,20000.000000,8.325003e+06
1,410,64,N,N,N,N,N,N,N,2,...,0.056333,0.045233,0.175067,0.363900,0.296767,75.1910,6.557400e+08,4.111113e+09,20000.000000,8.720726e+06
2,410,45,N,N,N,N,N,N,N,6,...,0.114050,0.030900,0.406450,0.003200,0.326300,58.4800,5.420000e+08,4.145011e+09,64114.242424,9.267336e+06
3,410,45,N,N,N,N,N,N,N,8,...,0.114050,0.030900,0.406450,0.003200,0.326300,59.5800,5.420000e+08,4.145011e+09,64114.242424,9.096222e+06
4,100,292,Y,Y,Y,Y,N,N,N,5,...,0.001233,0.301567,0.001233,0.001233,0.001200,59.7832,1.707400e+09,1.165011e+09,436558.904110,2.855560e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865,410,747,N,N,Y,N,Y,Y,Y,161,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.1316,4.252000e+08,4.139013e+09,30392.250000,5.053561e+06
1866,410,747,N,N,Y,N,Y,Y,Y,57,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,30392.250000,5.013486e+06
1867,410,747,N,N,Y,N,Y,Y,Y,57,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,30392.250000,5.013486e+06
1868,410,747,N,N,Y,N,Y,Y,Y,216,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3832,4.272000e+08,4.139013e+09,30392.250000,5.062193e+06


In [68]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42
)

In [69]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '공공주택지구',
       '대규모택지개발지구', '수도권내민영공공주택지구', '공급세대수', '순위', '거주지역', '접수건수', '경쟁률',
       '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5', '토픽 6', '토픽 7', '전용면적',
       '공급금액(최고가 기준)', '법정동코드', '거래금액(만원)', '전용면적당 시세차익'],
      dtype='object')
Index(['공급규모', '공급세대수', '접수건수', '경쟁률', '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5',
       '토픽 6', '토픽 7', '전용면적', '공급금액(최고가 기준)', '법정동코드', '거래금액(만원)',
       '전용면적당 시세차익', '투기과열지구_N', '투기과열지구_Y', '조정대상지역_N', '조정대상지역_Y',
       '분양가상한제_N', '분양가상한제_Y', '정비사업_N', '정비사업_Y', '공공주택지구_N', '공공주택지구_Y',
       '대규모택지개발지구_N', '대규모택지개발지구_Y', '수도권내민영공공주택지구_N', '수도권내민영공공주택지구_Y',
       '순위_1순위', '거주지역_기타지역', '거주지역_해당지역', '공급지역코드_100', '공급지역코드_400',
       '공급지역코드_410'],
      dtype='object')
⚠️ Warning: 새로운 법정동코드 발견 {'4146510300.0', '2871031025.0', '4150032026.0'}. 'unknown'으로 대체합니다.
Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '공공주택지구',
       '대규모택지개발지구', '수도권내민영공공주택지구', '공급세대수', '순위', '거주지

c:\Users\user\anaconda3\envs\venv\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\user\anaconda3\envs\venv\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [70]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1496 entries, 1539 to 1126
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   공급규모            1496 non-null   float64
 1   공급세대수           1496 non-null   float64
 2   접수건수            1496 non-null   float64
 3   경쟁률             1496 non-null   float64
 4   토픽 1            1496 non-null   float64
 5   토픽 2            1496 non-null   float64
 6   토픽 3            1496 non-null   float64
 7   토픽 4            1496 non-null   float64
 8   토픽 5            1496 non-null   float64
 9   토픽 6            1496 non-null   float64
 10  토픽 7            1496 non-null   float64
 11  전용면적            1496 non-null   float64
 12  공급금액(최고가 기준)    1496 non-null   float64
 13  법정동코드           1496 non-null   int64  
 14  거래금액(만원)        1496 non-null   float64
 15  전용면적당 시세차익      1496 non-null   float64
 16  투기과열지구_N        1496 non-null   bool   
 17  투기과열지구_Y        1496 non-null   boo

### 학습

In [1]:
# 최고당첨금액 , 최저당첨금액
'''
Randomforest, XGB, LGB 비교결과
XGB가 가장 높았지만, 성능평가에서 0.1정도의 아주 작은 차이라 더 가볍고 빠르게 돌아가는 모델인 LightGBM 이용 제안
추후 결정되면 코드 적어놓겠음
'''

'\nRandomforest, XGB, LGB 비교결과\nXGB가 가장 높았지만, 성능평가에서 0.1정도의 아주 작은 차이라 더 가볍고 빠르게 돌아가는 모델인 LightGBM 이용 제안\n추후 결정되면 코드 적어놓겠음\n'

### 모델 저장 

In [10]:
# 모델 저장 및 로드

version = 'rf_grid_0.0.1'

# 모델 저장
joblib.dump(rf_model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_rf_grid_0.0.1.pkl']

In [ ]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.2.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)